In [1]:
from hypster import HP, config


@config
def rag_config(hp: HP):
    from haystack import Pipeline
    from haystack.components.builders import PromptBuilder
    from haystack.components.generators import OpenAIGenerator
    from haystack.components.writers import DocumentWriter
    from haystack.document_stores.types import DuplicatePolicy

    template = """
    Given the following information, answer the question.

    Context:
    {% for document in documents %}
        {{ document.content }}
    {% endfor %}

    Question: {{question}}
    Answer:
    """
    prompt_builder = PromptBuilder(template=template)

    emb_type = hp.select(["sentence-transformers", "fastembed"], default="sentence-transformers")
    if emb_type == "sentence-transformers":
        from haystack.components.embedders import SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder
        from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
        from haystack.document_stores.in_memory import InMemoryDocumentStore

        model_types = ["sentence-transformers/all-MiniLM-L12-v2", "sentence-transformers/paraphrase-MiniLM-L6-v2"]
        emb_model = hp.select(model_types, default="sentence-transformers/paraphrase-MiniLM-L6-v2")

        document_store = InMemoryDocumentStore()
        doc_embedder = SentenceTransformersDocumentEmbedder(model=emb_model, batch_size=hp.int(16, min=2, max=256))
        text_embedder = SentenceTransformersTextEmbedder(model=emb_model)
        retriever = InMemoryEmbeddingRetriever(document_store, top_k=hp.int(10))

        embedding_str = "embedding"

    else:
        from fastembed import SparseTextEmbedding
        from haystack_integrations.components.embedders.fastembed import (
            FastembedSparseDocumentEmbedder,
            FastembedSparseTextEmbedder,
        )
        from haystack_integrations.components.retrievers.qdrant import QdrantSparseEmbeddingRetriever
        from haystack_integrations.document_stores.qdrant import QdrantDocumentStore

        models = [dct["model"] for dct in SparseTextEmbedding.list_supported_models()]
        document_store = QdrantDocumentStore(":memory:", recreate_index=True, use_sparse_embeddings=True)

        emb_model = hp.select(models)
        doc_embedder = FastembedSparseDocumentEmbedder(model=emb_model, batch_size=hp.int(16, min=2, max=256))
        retriever = QdrantSparseEmbeddingRetriever(document_store=document_store)
        text_embedder = FastembedSparseTextEmbedder(model=emb_model)

        embedding_str = "sparse_embedding"

    generator = OpenAIGenerator(
        model=hp.select(["gpt-4o", "gpt-4o-mini"], default="gpt-4o-mini"),
        generation_kwargs={"temperature": hp.number(0, min=0, max=1, name="temperature")},
    )

    document_writer = DocumentWriter(document_store, policy=DuplicatePolicy.OVERWRITE)

    indexing_pipeline = Pipeline()
    indexing_pipeline.add_component("doc_embedder", doc_embedder)
    indexing_pipeline.add_component("document_writer", document_writer)
    indexing_pipeline.connect("doc_embedder", "document_writer")

    retrieval_pipeline = Pipeline()
    retrieval_pipeline.add_component("text_embedder", text_embedder)
    retrieval_pipeline.add_component("retriever", retriever)
    retrieval_pipeline.add_component("prompt_builder", prompt_builder)
    retrieval_pipeline.add_component("llm", generator)

    retrieval_pipeline.connect(f"text_embedder.{embedding_str}", f"retriever.query_{embedding_str}")
    retrieval_pipeline.connect("retriever", "prompt_builder.documents")
    retrieval_pipeline.connect("prompt_builder", "llm")


In [5]:
from hypster.ui import apply_vscode_theme, interactive_config

apply_vscode_theme()
results = interactive_config(rag_config)

## Indexing

In [3]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset][:10]
indexing_pipeline = results["indexing_pipeline"]
indexing_pipeline.warm_up()

indexing_pipeline.run({"doc_embedder": {"documents": docs}})

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'document_writer': {'documents_written': 10}}

## Response

In [4]:
retrieval_pipeline = results["retrieval_pipeline"]
question = "What does Rhodes Statue look like?"

response = retrieval_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(response["llm"]["replies"][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Colossus of Rhodes was a massive statue of the Greek sun-god Helios, standing approximately 70 cubits (about 33 meters or 108 feet) high. It was constructed with iron tie bars and brass plates that formed its skin, while the interior was filled with stone blocks. The statue was depicted in a standing position, with its feet carved in stone and covered with bronze plates. The design included eight forged iron bars that supported the ankles and legs, and the statue's fingers were notably large, with some accounts stating that a person could not fully clasp the thumb with their arms.

The statue was originally placed on a 15-meter-high (49-foot) white marble pedestal near the entrance of Rhodes harbor. Its impressive height and the grandeur of its construction made it one of the tallest statues of the ancient world. Although it collapsed during an earthquake in 226 BC, the remains were still described as awe-inspiring, with vast caverns visible in the interior where the limbs had brok